In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
import os
import re

In [2]:
import new_ias_basic_function as ias
from settings.config import login_b, login_a, pass_b, pass_a
from settings.config import *
name_period = '2024'

In [3]:
#========= Вход
ias.come_application(login_b, pass_b)
#======== Чистка фильтров
ias.clear_filters('мероприятия')
#======== Фильтр календаря
ias.filter_application(name_period, 'мероприятия')
#======== Фильтр статуса
ias.status_sport()
#======== Фильтр участия на соглосование
ias.status_paty()
#======= Цикл перебора мероприятий

In [4]:
driver.implicitly_wait(30)
find = driver.find_element(By.XPATH,  
         "//app-events-section-header/form[1]/div[2]/div[2]/app-search-input[1]/div[1]/input[1]")
find.clear()
time.sleep(1)
find.send_keys(91443, Keys.RETURN)
driver.implicitly_wait(30)
time.sleep(1)
driver.find_element(By.XPATH, "//datatable-body-cell/div[1]/div[1]/img[1]").click()
driver.implicitly_wait(30)
time.sleep(1)

In [5]:
time.sleep(1)
driver.find_element(By.XPATH, "//div[contains(text(),'Развернуть')]").click()
time.sleep(1)

In [6]:
handles = driver.window_handles
driver.switch_to.window(handles[1])
time.sleep(5)

In [7]:
driver.find_element(By.XPATH, "//mat-tab-header/div[1]/div[1]/div[1]/div[3]/div[1]/div[1]/div[1]").click()
time.sleep(5)
driver.implicitly_wait(50)
elements_button = driver.find_elements(By.CLASS_NAME, "mat-menu-trigger")

In [8]:
#elements_button = driver.find_elements(By.CSS_SELECTOR, "span.icon-hamburger_outline")
elements_text = driver.find_elements(By.XPATH, "//tbody[@class='ng-star-inserted']")
print(len(elements_button))
text =''
for i in elements_text:
    text += i.text +"\n"
    pattern = re.compile(r'\(утверждена\)|\(на согласование\)|\(проект\)|\(архив\)')
    #values_in_brackets = re.findall(r'\((.*?)\)', text)
    values_in_brackets =re.findall(pattern, text)
#while True:
for idx, value in enumerate(values_in_brackets, 0):
    print(idx, value)
    if value == '(на согласование)':
        el_status = idx
        driver.execute_script("window.scrollTo(0, 5000)")
        time.sleep(1)
        driver.implicitly_wait(50)
        print(el_status)
        elements_button[el_status].click()

        button = driver.find_element(By.LINK_TEXT,'Редактировать статус участия')
        button.click()
        driver.implicitly_wait(30)
        input_status = driver.find_element(By.XPATH, '//input[@aria-autocomplete="list"]')
        print('Элемент найден!')
        driver.implicitly_wait(55)
        time.sleep(2)
        try:
            input_status.click()
            time.sleep(1)
            try:
                accept = driver.find_element(By.XPATH, "//span[contains(text(),'утверждена')]")
                accept.click()
                time.sleep(1)
                driver.implicitly_wait(15)
                accept = driver.find_element(By.XPATH, "//button[contains(text(),'Сохранить')]")
                accept.click()
                print('Участие соглосованно')
            except:
                cancel = driver.find_element(By.XPATH, "//button[contains(text(),'Отменить')]")
                cancel.click()
                pass
        except:
            print(el_status, ' Ошибка соглосования элемента!')
            pass
    else:
        pass

   

2
0 (на согласование)
0
Элемент найден!
Участие соглосованно
1 (на согласование)
1
Элемент найден!
Участие соглосованно
